# AWS Postgres RDS Debug Notebook

This notebook allows to run queries manually against the AWS postgres DB.

In [1]:
import sys
sys.path.insert(0, '/Users/ferdi/Documents/agent-copilot/src')

import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from aco.server.database_manager import DB
DB.switch_mode("remote")

2025-11-30 17:09:58,287 - ACO - INFO - DatabaseManager initialized with backend: local
2025-11-30 17:09:58,298 - ACO - INFO - Switched to remote PostgreSQL database


## List Table Entries

### experiments table

In [7]:
# Get all experiments
experiments = DB.query_all(
    "SELECT session_id, parent_session_id, name, timestamp FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

Found 4 experiments:


,0,1,2,3
0,fc7b58ae-3603-4d20-a555-1f672b5335cc,fc7b58ae-3603-4d20-a555-1f672b5335cc,test_api_calls_Anthropic_messages_create_remot...,2025-11-30 17:31:18.254303
1,fdc94f5f-7995-45d5-b4f7-8f4cbdda8649,fdc94f5f-7995-45d5-b4f7-8f4cbdda8649,test_api_calls_OpenAI_responses_create_remote_...,2025-11-30 17:30:47.119709
2,2b4d6eb8-7760-480e-a4e8-c081c9d1a784,2b4d6eb8-7760-480e-a4e8-c081c9d1a784,test_api_calls_Anthropic_messages_create_remot...,2025-11-30 17:29:51.960360
3,05ca4002-ba7a-4fef-ba04-dc27ee2f27e4,05ca4002-ba7a-4fef-ba04-dc27ee2f27e4,test_api_calls_Anthropic_messages_create_remot...,2025-11-30 17:29:17.367247


### llm_calls table

In [40]:
# Get recent LLM calls
llm_calls = DB.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

2025-11-19 15:38:22,346 - ACO - INFO - [QUERY_ALL] START thread=140704361521024 sql=SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20...
2025-11-19 15:38:22,347 - ACO - INFO - [CONN] GET conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:226
2025-11-19 15:38:22,389 - ACO - INFO - [QUERY_ALL] SUCCESS thread=140704361521024 conn=5044824304 rows=0
2025-11-19 15:38:22,389 - ACO - INFO - [CONN] RETURN conn=5044824304 thread=140704361521024 caller=postgres.py:query_all:238


No LLM calls found


## Clear tables

Here's a function to clear all records from the experiments table:

In [4]:
def clear_experiments_table():
    """Clear all records from the experiments table (deletes all records)"""
    try:
        # Execute DELETE query to remove all records
        DB.execute("DELETE FROM llm_calls")
        DB.execute("DELETE FROM attachments")
        DB.execute("DELETE FROM experiments")

        print("✅ Successfully cleared all records from experiments table")
        
        # Verify the table is empty
        count = DB.query_one("SELECT COUNT(*) as count FROM experiments")
        print(f"   Remaining records: {count['count']}")
        
    except Exception as e:
        print(f"❌ Failed to clear experiments table: {e}")
        
# Call the function to clear the table
# Uncomment the line below to actually run it:
clear_experiments_table()

✅ Successfully cleared all records from experiments table
   Remaining records: 0


# Drop tables

In [ ]:
# # FRESH CONNECTION + EMERGENCY RESET (All-in-one)
# print("🚨 Creating fresh connection and performing emergency reset...")
# from aco.common.constants import REMOTE_DATABASE_URL
# import psycopg2
# import psycopg2.extras
# from urllib.parse import urlparse

# if REMOTE_DATABASE_URL:
#     parsed = urlparse(REMOTE_DATABASE_URL)
    
#     try:
#         # Create a fresh connection
#         print("1️⃣ Establishing fresh connection...")
#         fresh_conn = psycopg2.connect(
#             host=parsed.hostname,
#             port=parsed.port or 5432,
#             user=parsed.username,
#             password=parsed.password,
#             database=parsed.path[1:],
#             connect_timeout=15
#         )
#         fresh_conn.autocommit = True  # Auto-commit for immediate effect
#         cursor = fresh_conn.cursor()
#         print("   ✅ Fresh connection established")
        
#         # Kill all other connections
#         print("\n2️⃣ Killing hanging sessions...")
#         cursor.execute("SELECT current_database()")
#         current_db = cursor.fetchone()[0]
        
#         cursor.execute("""
#             SELECT pg_terminate_backend(pid) as killed
#             FROM pg_stat_activity
#             WHERE datname = %s 
#             AND pid != pg_backend_pid()
#             AND state != 'idle'
#         """, (current_db,))
#         killed_sessions = cursor.fetchall()
#         print(f"   ✅ Terminated {len(killed_sessions)} hanging sessions")
        
#         # Force drop all tables (uncomment below)
#         print("\n3️⃣ Force dropping tables...")
#         cursor.execute("DROP TABLE IF EXISTS llm_calls CASCADE")
#         cursor.execute("DROP TABLE IF EXISTS attachments CASCADE")
#         cursor.execute("DROP TABLE IF EXISTS experiments CASCADE")
#         print("   ✅ All tables dropped")
        
#         # Recreate experiments table
#         print("\n4️⃣ Recreating tables with BYTEA schema...")
#         cursor.execute("""
#             CREATE TABLE experiments (
#                 session_id TEXT PRIMARY KEY,
#                 parent_session_id TEXT,
#                 graph_topology TEXT,
#                 color_preview TEXT,
#                 timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#                 cwd TEXT,
#                 command TEXT,
#                 environment TEXT,
#                 code_hash TEXT,
#                 name TEXT,
#                 success TEXT CHECK (success IN ('', 'Satisfactory', 'Failed')),
#                 notes TEXT,
#                 log TEXT,
#                 FOREIGN KEY (parent_session_id) REFERENCES experiments (session_id),
#                 UNIQUE (parent_session_id, name)
#             )
#         """)
#         print("   ✅ Experiments table created")
        
#         # Recreate llm_calls table with BYTEA output (THE CRITICAL FIX!)
#         cursor.execute("""
#             CREATE TABLE llm_calls (
#                 session_id TEXT,
#                 node_id TEXT,
#                 input BYTEA,
#                 input_hash TEXT,
#                 input_overwrite BYTEA,
#                 output BYTEA,
#                 color TEXT,
#                 label TEXT,
#                 api_type TEXT,
#                 timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#                 PRIMARY KEY (session_id, node_id)
#             )
#         """)
#         # HACK: Remove foreign key constraint because things can happen in parallel.
#         print("   ✅ LLM calls table created with BYTEA output column")
        
#         # Recreate attachments table
#         cursor.execute("""
#             CREATE TABLE attachments (
#                 file_id TEXT PRIMARY KEY,
#                 session_id TEXT,
#                 line_no INTEGER,
#                 content_hash TEXT,
#                 file_path TEXT,
#                 taint TEXT,
#                 FOREIGN KEY (session_id) REFERENCES experiments (session_id)
#             )
#         """)
#         print("   ✅ Attachments table created")
        
#         # Create indexes
#         cursor.execute("CREATE INDEX attachments_content_hash_idx ON attachments(content_hash)")
#         cursor.execute("CREATE INDEX original_input_lookup ON llm_calls(session_id, input_hash)")
#         cursor.execute("CREATE INDEX experiments_timestamp_idx ON experiments(timestamp DESC)")
#         print("   ✅ Indexes created")
        
#         # Verify the schema fix
#         cursor.execute("""
#             SELECT data_type 
#             FROM information_schema.columns 
#             WHERE table_name = 'llm_calls' AND column_name = 'output'
#         """)
#         output_type = cursor.fetchone()[0]
        
#         print(f"\n🎉 SUCCESS! Schema completely rebuilt")
#         print(f"   ✅ Output column is now: {output_type}")
#         print(f"   ✅ All tables recreated cleanly")
#         print(f"   ✅ All locks cleared")
                    
#         fresh_conn.close()
        
#     except Exception as e:
#         print(f"❌ Emergency reset failed: {e}")
#         print(f"   Error type: {type(e).__name__}")
        
#         if "timeout" in str(e).lower():
#             print("   → Connection still timing out. Database may be overloaded.")
#         elif "permission" in str(e).lower():
#             print("   → Permission denied. Check database user privileges.")
#         else:
#             print(f"   → Unexpected error: {e}")
            
# else:
#     print("❌ No DATABASE_URL found")

🚨 Creating fresh connection and performing emergency reset...
1️⃣ Establishing fresh connection...
   ✅ Fresh connection established

2️⃣ Killing hanging sessions...
   ✅ Terminated 0 hanging sessions

3️⃣ Force dropping tables...
   ✅ All tables dropped

4️⃣ Recreating tables with BYTEA schema...
   ✅ Experiments table created
   ✅ LLM calls table created with BYTEA output column
   ✅ Attachments table created
   ✅ Indexes created

🎉 SUCCESS! Schema completely rebuilt
   ✅ Output column is now: bytea
   ✅ All tables recreated cleanly
   ✅ All locks cleared
